In [133]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, BayesianRidge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [3]:
"""
Read in train and test as Pandas DataFrames
"""
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [18]:
df_test_full = df_test
df_train_full = df_train

In [248]:
df_test = df_train_full.sample(n=100000)
df_train = df_train_full.sample(n=100000)

In [249]:
df_train.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,gap
253124,C1=Cc2c(C1)c1ncc3cc(cnc3c1c1=CCC=c21)-c1scc2cc...,1,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1.79
961291,c1ccc(cc1)-c1ccc2c(c1)sc1c2c2=C[SiH2]C=c2c2ccc...,1,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1.53
455656,c1cc2ncc3cc4cc(sc4cc3c2c2c[nH]cc12)-c1cccc2=CC...,1,0,0,0,1,1,1,0,0,...,1,0,0,0,0,0,0,0,0,1.84
814199,[nH]1c(cc2cnc3c4sccc4c4cscc4c3c12)-c1cncc2nsnc12,0,0,0,0,1,1,1,0,0,...,1,0,0,0,0,0,0,0,0,1.37
574823,c1sc(-c2sc(-c3ccccn3)c3sccc23)c2nccnc12,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1.51


In [250]:
df_test.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,gap
867846,[nH]1cccc1-c1[SiH2]c(cc1)-c1ccc(cc1)-c1scc2cc[...,1,0,0,0,1,1,1,0,0,...,1,0,0,1,0,0,0,0,0,1.80
79322,c1sc(-c2sc(c3[SiH2]ccc23)-c2ccc(cc2)-c2nccc3ns...,1,0,0,0,1,1,1,0,0,...,1,0,0,0,0,0,0,0,0,1.24
806798,c1cc2c3nsnc3c3c4[SiH2]C(=Cc4ccc3c2s1)c1cncc2ns...,1,0,0,0,1,0,1,0,0,...,1,0,0,1,0,0,0,0,0,1.82
232257,[nH]1c2ccncc2c2cc3c4c[nH]cc4c(cc3cc12)-c1nccs1,0,0,0,0,1,1,1,0,0,...,1,0,0,0,0,0,0,0,0,2.32
393507,c1sc(-c2cnc(s2)-c2ccc(-c3scc4nccnc34)c3cocc23)...,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1.45


In [251]:
test_ids = df_test.index
test_gap = df_test.gap

In [252]:
#store gap values
Y_train = df_train.gap.values
#row where testing examples start
test_idx = df_train.shape[0]
#delete 'Id' column
# df_test = df_test.drop(['Unnamed: 0'], axis=1)
#delete 'gap' column
df_train = df_train.drop(['gap'], axis=1)
df_test = df_test.drop(['gap'], axis=1)

In [253]:
#DataFrame with all train and test examples so we can more easily apply feature engineering on
df_all = pd.concat((df_train, df_test), axis=0)
df_all.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
253124,C1=Cc2c(C1)c1ncc3cc(cnc3c1c1=CCC=c21)-c1scc2cc...,1,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
961291,c1ccc(cc1)-c1ccc2c(c1)sc1c2c2=C[SiH2]C=c2c2ccc...,1,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
455656,c1cc2ncc3cc4cc(sc4cc3c2c2c[nH]cc12)-c1cccc2=CC...,1,0,0,0,1,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0
814199,[nH]1c(cc2cnc3c4sccc4c4cscc4c3c12)-c1cncc2nsnc12,0,0,0,0,1,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0
574823,c1sc(-c2sc(-c3ccccn3)c3sccc23)c2nccnc12,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


In [254]:
"""
Example Feature Engineering

this calculates the length of each smile string and adds a feature column with those lengths
Note: this is NOT a good feature and will result in a lower score!
"""
#smiles_len = np.vstack(df_all.smiles.astype(str).apply(lambda x: len(x)))
#df_all['smiles_len'] = pd.DataFrame(smiles_len)


'\nExample Feature Engineering\n\nthis calculates the length of each smile string and adds a feature column with those lengths\nNote: this is NOT a good feature and will result in a lower score!\n'

In [255]:
#Drop the 'smiles' column
df_all = df_all.drop(['smiles'], axis=1)
vals = df_all.values
X_train = vals[:test_idx]
X_test = vals[test_idx:]
print "Train features:", X_train.shape
print "Train gap:", Y_train.shape
print "Test features:", X_test.shape

Train features: (100000, 256)
Train gap: (100000,)
Test features: (100000, 256)


In [256]:
LR = LinearRegression()
LR.fit(X_train, Y_train)
LR_pred = LR.predict(X_test)

In [257]:
RF = RandomForestRegressor()
RF.fit(X_train, Y_train)
RF_pred = RF.predict(X_test)

In [258]:
RF_100 = RandomForestRegressor(n_estimators=100)
RF_100.fit(X_train, Y_train)
RF_pred_100 = RF_100.predict(X_test)

In [259]:
# Bayesian Ridge Regression
BRR = BayesianRidge(compute_score=True)
BRR.fit(X_train, Y_train)
BRR_pred = BRR.predict(X_test)

In [260]:
# Lasso Regression
L = Lasso(alpha=0.1)
L.fit(X_train, Y_train)
L_pred = L.predict(X_test)

### Comparing apples to apples for ensemble

In [261]:
ensemble_train = df_train.drop(['smiles'], axis=1)
X_train_ensemble = ensemble_train.values
ensemble_train['LR_pred'] = LR.predict(X_train_ensemble)
ensemble_train['RF_pred_100'] = RF_100.predict(X_train_ensemble)
ensemble_train['BRR_pred'] = BRR.predict(X_train_ensemble)
ensemble_train['L_pred'] = L.predict(X_train_ensemble)

In [262]:
# get new X train with the prediction values
X_train_ensemble = ensemble_train.values

In [263]:
ensemble_test = df_test.drop(['smiles'], axis=1)
X_test_ensemble = ensemble_test.values
ensemble_test['LR_pred'] = LR.predict(X_test_ensemble)
ensemble_test['RF_pred_100'] = RF_100.predict(X_test_ensemble)
ensemble_test['BRR_pred'] = BRR.predict(X_test_ensemble)
ensemble_test['L_pred'] = L.predict(X_test_ensemble)
# get new X test with the prediction values
X_test_ensemble = ensemble_test.values

In [264]:
LR_ensemble = LinearRegression()
LR_ensemble.fit(X_train_ensemble, Y_train)
LR_ensemble_pred = LR_ensemble.predict(X_test_ensemble)

RF_ensemble = RandomForestRegressor(n_estimators=100)
RF_ensemble.fit(X_train_ensemble, Y_train)
RF_ensemble_pred = RF_ensemble.predict(X_test_ensemble)

In [271]:
# Bayesian Ridge Regression
BRR_ensemble = BayesianRidge(compute_score=True)
BRR_ensemble.fit(X_train_ensemble, Y_train)
BRR_ensemble_pred = BRR_ensemble.predict(X_test_ensemble)

In [274]:
# Lasso Regression
L_ensemble = Lasso(alpha=0.1)
L_ensemble.fit(X_train_ensemble, Y_train)
L_ensemble_pred = L_ensemble.predict(X_test_ensemble)

### Evaluation of these Algorithms

In [265]:
# baselines Linear
LR_rms = math.sqrt(mean_squared_error(LR_pred, test_gap))
print "LR_rms"
print LR_rms

#baselines Random Forest
RF_rms = math.sqrt(mean_squared_error(RF_pred, test_gap))
print "RF_rms"
print RF_rms

LR_rms
0.299570007751
RF_rms
0.275082215811


In [266]:
BRR_rms = math.sqrt(mean_squared_error(BRR_pred, test_gap))
print "BRR_rms"
print BRR_rms

BRR_rms
0.299563390761


In [267]:
# lasso regression
L_rms = math.sqrt(mean_squared_error(L_pred, test_gap))
print "L_rms"
print L_rms

L_rms
0.40840438825


In [268]:
# random forest with 100
RF_100_rms = math.sqrt(mean_squared_error(RF_pred_100, test_gap))
print "RF_100_rms"
print RF_100_rms

RF_100_rms
0.274914290212


In [277]:
# ensemble methodx
# random forest with 100
RF_ensemble_rms = math.sqrt(mean_squared_error(RF_ensemble_pred, test_gap))
print "RF_ensemble_rms"
print RF_ensemble_rms
# linear ensemble
LR_ensemble_rms = math.sqrt(mean_squared_error(LR_ensemble_pred, test_gap))
print "LR_ensemble_rms"
print LR_ensemble_rms
# bayesian ensemble
BRR_ensemble_rms = math.sqrt(mean_squared_error(BRR_ensemble_pred, test_gap))
print "BRR_ensemble_rms"
print BRR_ensemble_rms
# lasso ensemble
L_ensemble_rms = math.sqrt(mean_squared_error(L_ensemble_pred, test_gap))
print "L_ensemble_rms"
print L_ensemble_rms

RF_ensemble_rms
0.275113725376
LR_ensemble_rms
0.274943867533
BRR_ensemble_rms
0.274927986465
L_ensemble_rms
0.40840438825


# Why do linear and BRR outperform RF in this case? Maybe RF had too high variance for this?

In [281]:
params = LR_ensemble.coef_

In [284]:
params

array([  4.42444378e+07,  -1.95058053e+12,  -1.38558148e+12,
         2.25499514e+12,   3.82361196e+08,   4.03375299e+10,
         2.83021809e+08,  -1.62653366e+11,  -6.14707323e+11,
         1.86833093e+12,   2.83126232e+11,  -2.95404314e+12,
        -2.28141956e+12,  -6.01581161e+11,   1.59291832e+12,
        -3.86398288e+12,   1.47795058e+12,  -6.77790848e+11,
         3.73573590e+11,   1.28637902e+12,  -1.26465385e+12,
        -9.65515747e+11,  -1.02577936e+12,   1.34299933e+12,
        -1.08067147e+08,  -4.94906910e+10,   2.74680378e+12,
        -9.61558698e+11,  -6.41537533e+10,   2.29208503e+11,
        -2.24975905e+12,   2.34078846e+11,  -1.45196887e+12,
        -8.48244011e-04,   5.24092190e-04,   5.79958978e-04,
         3.56118132e+08,   3.49460599e-04,   1.30439679e-04,
         1.04755903e-04,  -2.50456897e-06,   4.98008060e-04,
        -1.73099591e-04,   5.57556486e+08,  -3.56765783e-05,
         1.95754863e-05,   1.69928204e-05,  -4.85320161e-07,
         8.94611807e-08,

## Ensemble methods 
Now create a new training set with columns as follows: id, linear, random forest with 100, bayesian ridge regression, lasso regression, and all the other features. We also have to create a test set for this.

### Can try adding all the other features back in - THIS IMPLEMENTATION DOESN'T WORK

#### Creating the training set

In [200]:
test_ids

Int64Index([343852, 972179,   8904, 166852, 558603, 111403, 318809, 806625,
            616963, 720639, 
            ...
            197959, 385059, 340766, 926646, 686870, 621446, 931684, 500846,
            454584, 472244],
           dtype='int64', length=100000)

In [201]:
ensemble_training = pd.DataFrame({'linear' : LR_pred,'random_forest': RF_pred_100,
                           'bayesian_ridge' : BRR_pred,
                           'lasso' : L_pred})

In [202]:
ensemble_training.head(5)

,bayesian_ridge,lasso,linear,random_forest
0,2.069898,1.913873,2.070801,2.147891
1,1.764883,1.913873,1.762695,1.721839
2,1.749688,1.913873,1.750244,1.834650
3,2.059840,1.913873,2.060791,1.963281
4,1.764883,1.913873,1.762695,1.721839


In [207]:
ensemble_training = df_test.drop(['smiles'], axis=1)
ensemble_training['linear'] = LR_pred
ensemble_training['random_forest'] = RF_pred_100
ensemble_training['bayesian_ridge'] = BRR_pred
ensemble_training['lasso'] = L_pred

In [208]:
Y_train = test_gap # actual results for this training set

### 260 features

In [209]:
vals = ensemble_training.values
X_train_ensemble = vals[:test_idx]
print "Train features:", X_train_ensemble.shape
print "Train gap:", Y_train.shape

Train features: (100000, 260)
Train gap: (100000,)


#### Creating the test set
We need to take a new random sample from our original training and come up with predictions

In [226]:
df_test_ensemble = df_train_full.sample(n=100000)
ensemble_test_ids = df_test_ensemble.index
ensemble_test_gap = df_test_ensemble.gap
df_test_ensemble = df_test_ensemble.drop(['gap'], axis=1)

In [227]:
#Drop the 'smiles' column
ensemble_testing = df_test_ensemble.drop(['smiles'], axis=1)
X_test_ensemble = ensemble_testing.values

In [228]:
LR_pred_test = LR.predict(X_test_ensemble)
BRR_pred_test = BRR.predict(X_test_ensemble)
RF_pred_100_test = RF_100.predict(X_test_ensemble)
L_pred_test = L.predict(X_test_ensemble)

In [229]:
ensemble_testing['linear'] = LR_pred_test
ensemble_testing['random_forest'] = RF_pred_100_test
ensemble_testing['bayesian_ridge'] = BRR_pred_test
ensemble_testing['lasso'] = L_pred_test

In [231]:
X_test_ensemble = ensemble_testing.values

In [234]:
len(X_test_ensemble), len(X_train_ensemble)

(100000, 100000)

#### Do random forest again

In [235]:
LR_ensemble = LinearRegression()
LR_ensemble.fit(X_train_ensemble, Y_train)
LR_ensemble_pred = LR_ensemble.predict(X_test_ensemble)

RF_ensemble = RandomForestRegressor(n_estimators=100)
RF_ensemble.fit(X_train_ensemble, Y_train)
RF_ensemble_pred = RF_ensemble.predict(X_test_ensemble)

#### Evaluate

In [236]:
# baselines Linear
LR_rms = math.sqrt(mean_squared_error(LR_pred, ensemble_test_gap))
print "LR_rms"
print LR_rms

#baselines Random Forest
RF_rms = math.sqrt(mean_squared_error(RF_pred, ensemble_test_gap))
print "RF_rms"
print RF_rms

LR_rms
8016173212.42
RF_rms
0.510969590839


So for some reason this is even worse :( Maybe should keep all the old features as well...

### Save results to CSV

In [60]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

In [61]:
write_to_file("sample1.csv", LR_pred)
write_to_file("sample2.csv", RF_pred)
write_to_file("bayesianridge.csv", BRR_pred)
write_to_file("randomforest100trees.csv", RF_pred_100)